In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow import keras
import keras
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import MeanSquaredError

import numpy as np
import pandas as pd
import cvxpy as cp
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
cwd = os.getcwd()

def make_dir(path):
    if os.path.exists(path) is False:
        os.makedirs(path)

def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    print('MAE: {:.2f}'.format(mae))
    print('RMSE: {:.2f}'.format(rmse))
    print('')
    print('')
    return mae, rmse

In [4]:
sl = pd.read_csv('dataset/ercot_sl_2019_2023.csv')
sl['sced_time_stamp_local'] = pd.to_datetime(sl['sced_time_stamp_local'])
sl.set_index('sced_time_stamp_local', inplace=True)
sl = sl.resample('h').mean()
date_range = pd.date_range(start=sl.index.min(), end=sl.index.max(), freq='h')
sl = sl[~sl.index.duplicated(keep='first')]
sl = sl.reindex(date_range, fill_value=np.nan)
sl.interpolate(method='time', inplace=True)


In [5]:
dap = pd.read_csv('dataset/hourly_ercot_day_ahead_sl_2019_2023.csv')
dap['timestamp'] = pd.to_datetime(dap['timestamp'])
dap.set_index('timestamp', inplace=True)
date_range = pd.date_range(start=dap.index.min(), end='2024-01-01 23:55:00', freq='h')
dap = dap[~dap.index.duplicated(keep='first')]
dap = dap.reindex(date_range, fill_value=np.nan)
dap.interpolate(method='time', inplace=True)

C:\Users\natha\AppData\Local\Temp\ipykernel_14692\3109986249.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dap['timestamp'] = pd.to_datetime(dap['timestamp'])


In [6]:
forecasted_tlf = pd.read_csv('dataset/FORECASTED_TLF.csv')
forecasted_tlf['datetime'] = pd.to_datetime(forecasted_tlf['datetime'])
forecasted_tlf.set_index('datetime', inplace=True)
date_range = pd.date_range(start=forecasted_tlf.index.min(), end='2024-01-01 23:55:00', freq='h')
forecasted_tlf = forecasted_tlf[~forecasted_tlf.index.duplicated(keep='first')]
forecasted_tlf = forecasted_tlf.reindex(date_range, fill_value=np.nan)
forecasted_tlf.interpolate(method='time', inplace=True)

forecasted_dlf = pd.read_csv('dataset/FORECASTED_DLF.csv')
forecasted_dlf['datetime'] = pd.to_datetime(forecasted_dlf['datetime'])
forecasted_dlf.set_index('datetime', inplace=True)
date_range = pd.date_range(start=forecasted_dlf.index.min(), end='2024-01-01 23:55:00', freq='h')
forecasted_dlf = forecasted_dlf[~forecasted_dlf.index.duplicated(keep='first')]
forecasted_dlf = forecasted_dlf.reindex(date_range, fill_value=np.nan)
forecasted_dlf.interpolate(method='time', inplace=True)

In [7]:
#prepare the data for training and testing
df = pd.concat([dap, sl, forecasted_dlf, forecasted_tlf], axis=1)
df.columns = ['DAP', 'SCED', 'F_DLF', 'F_TLF']
df.dropna(inplace=True)

df

,DAP,SCED,F_DLF,F_TLF
2019-01-02 00:00:00,23.9250,26.216109,0.039835,0.016499
2019-01-02 01:00:00,23.3140,25.671136,0.031644,0.016441
2019-01-02 02:00:00,23.3475,24.970952,0.050937,0.016411
2019-01-02 03:00:00,23.0595,24.787712,0.015945,0.016441
2019-01-02 04:00:00,25.2672,24.887786,0.059952,0.016534
...,...,...,...,...
2023-12-31 19:00:00,19.3715,14.422107,0.012525,0.021356
2023-12-31 20:00:00,16.7739,12.597410,0.032945,0.021367
2023-12-31 21:00:00,15.0035,10.170172,0.033003,0.021383
2023-12-31 22:00:00,15.7699,9.872549,0.027504,0.021408


In [8]:

## log transform
log_data = df.copy(deep=True)
log_data.loc[:,"SCED"] = np.log(df.loc[:,"SCED"] + 1 - min(df.loc[:,"SCED"]))
log_data.loc[:,"DAP"] = np.log(df.loc[:,"DAP"] + 1 - min(df.loc[:,"DAP"]))
log_data.loc[:,"F_TLF"] = np.log(df.loc[:,"F_TLF"] + 1 - min(df.loc[:,"F_TLF"]))
log_data.loc[:,"F_DLF"] = np.log(df.loc[:,"F_DLF"] + 1 - min(df.loc[:,"F_DLF"]))

#Shift for prediction
df['DAP'] = df['DAP'].shift(-24)

# 3 years training, 1 year validation, 1 year testing
x_train_df_reg = log_data.loc[:'2021-12-31 23:55:00'].iloc[:,:]
x_val_df_reg = log_data.loc['2022-01-01 00:00:00':'2022-12-31 23:55:00'].iloc[:,:]
x_test_df_reg = log_data.loc['2023-01-01 00:00:00':].iloc[:,:]

# Shift back for target variable
df['DAP'] = df['DAP'].shift(24)

#Change target variable to DAP
y_train_df_reg = log_data.loc[:'2021-12-31 23:55:00'].iloc[:, :1]
y_val_df_reg = log_data.loc['2022-01-01 00:00:00':'2022-12-31 23:55:00'].iloc[:, :1]
y_test_df_reg = log_data.loc['2023-01-01 00:00:00':].iloc[:, :1]

x_train_df_reg.reset_index(drop=True, inplace=True)
x_val_df_reg.reset_index(drop=True, inplace=True)
x_test_df_reg.reset_index(drop=True, inplace=True)

# Standardization
x_mean_reg, x_std_reg = x_train_df_reg.mean(), x_train_df_reg.std()
y_mean_reg, y_std_reg = y_train_df_reg.mean(), y_train_df_reg.std()

x_std_reg = x_std_reg +0.00001

x_train_reg = (x_train_df_reg - x_mean_reg)/x_std_reg
x_val_reg = (x_val_df_reg - x_mean_reg)/x_std_reg
x_test_reg = (x_test_df_reg - x_mean_reg)/x_std_reg

y_train_reg = (y_train_df_reg - y_mean_reg)/y_std_reg
y_val_reg = (y_val_df_reg - y_mean_reg)/y_std_reg
y_test_reg = (y_test_df_reg - y_mean_reg)/y_std_reg

# Shift the data for the lags
n_steps_in = 24
n_steps_out = 24

x_train_lstm = np.array([x_train_reg[i:i+n_steps_in] for i in range(0, x_train_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_train_lstm = np.array([y_train_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_train_reg.shape[0]-n_steps_in-n_steps_out+1)])

x_val_lstm = np.array([x_val_reg[i:i+n_steps_in] for i in range(0, x_val_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_val_lstm = np.array([y_val_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_val_reg.shape[0]-n_steps_in-n_steps_out+1)])

x_test_lstm = np.array([x_test_reg[i:i+n_steps_in] for i in range(0, x_test_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_test_lstm = np.array([y_test_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_test_reg.shape[0]-n_steps_in-n_steps_out+1)])

print(x_train_lstm.shape,y_train_lstm.shape,x_val_lstm.shape,y_val_lstm.shape, x_test_lstm.shape,y_test_lstm.shape)

(26233, 24, 4) (26233, 24, 1) (8713, 24, 4) (8713, 24, 1) (8713, 24, 4) (8713, 24, 1)


In [9]:
# set hyperparameters
n_neurons  = 64  # number of neurons in the Dense layer
activation     = 'relu' # activation function
learning_rate  = 0.0005
minibatch_size = 64
num_epochs     = 50

# MLP model
lstm_model = Sequential()
lstm_model.add(LSTM(n_neurons,input_shape=(x_train_lstm.shape[1],x_train_lstm.shape[2]),
               return_sequences=True,activation=activation))
lstm_model.add(LSTM(n_neurons,return_sequences=False,
               activation=activation))
lstm_model.add(Dense(n_neurons,activation=activation))
lstm_model.add(Dense(y_train_lstm.shape[-2],activation='linear')) 

lstm_model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

lstm_model.summary()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = lstm_model.fit(x_train_lstm, y_train_lstm, 
                        validation_data = (x_val_lstm, y_val_lstm), 
                        batch_size = minibatch_size,
                        epochs = num_epochs,
                        verbose=1,
                        callbacks=[early_stop],
                        shuffle=False)

# saving trained model
model_path = os.path.join(cwd,'saved_model')
make_dir(model_path)
lstm_model.save(os.path.join(model_path,'DAP_Model_Week_3.h5'))


c:\Users\natha\OneDrive\Documents\ConcordNewEnergy\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 24, 64)         │        17,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │         1,560 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,408 (220.34 KB)

 Trainable params: 56,408 (220.34 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 1.1463 - val_loss: 0.9008
Epoch 2/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 8.3420 - val_loss: 1.2368
Epoch 3/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 1.0539 - val_loss: 0.9049
Epoch 4/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.6326 - val_loss: 0.8387
Epoch 5/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.5548 - val_loss: 7.0412
Epoch 6/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 1.1530 - val_loss: 0.8628
Epoch 7/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.6495 - val_loss: 0.8247
Epoch 8/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.5364 - val_loss: 0.7864
Epoch 9/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.4998 - val_loss: 0.7278
Epoch 10/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.5187 - val_loss: 0.7204
Epoch 11/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.4636 - val_loss: 0.6344
Epoch 12/50
410/410 ━━━━━━━━━━━━━━━━━━━━ 

In [10]:
# reload the saved model
model_path = os.path.join(cwd,'saved_model')
lstm_model = load_model(
    os.path.join(model_path, 'DAP_Model_Week_3.h5'),
    custom_objects={'mse': MeanSquaredError()}
)

# Make prediciton
y_test_pred = lstm_model.predict(x_test_lstm)

# Evaluate Prediction
evaluate_prediction(y_test_pred , y_test_lstm[:,:,0], 'lstm')

# Rescale to get values before normalization
y_test_pred_rescale = y_test_pred*y_std_reg.values + y_mean_reg.values
y_test_lstm_rescale = y_test_lstm*y_std_reg.values + y_mean_reg.values

# inverse log to get prices in the actual scale
y_test_pred_invlog = np.exp(y_test_pred_rescale) -1 + min(df.loc[:,"SCED"])
y_test_lstm_invlog = np.exp(y_test_lstm_rescale) -1 + min(df.loc[:,"SCED"])

# revaluation after the rescaling
evaluate_prediction(y_test_pred_invlog , y_test_lstm_invlog[:,:,0], 'lstm')

273/273 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
MAE: 0.56
RMSE: 1.00


MAE: 46.80
RMSE: 212.82




(np.float64(46.80489923633535), np.float64(212.8179825511515))